# Wykład 12: Projektowanie sieci ConvNet

Na tym wykładzie zaprojektujemy sami sieci ConvNet i prostymi zastosowaniami

Zobaczymy także trzy różne metody definiowania modeli w Kerasie:

- Sekewncyjny (Sequential)
- Funcjonalny (Functional)
- Zorientowany obietkowo (Object-Oriented)

Uwaga: Celem tego wykładu nie jest porównanie wydajności, ale tylko samych architektur

### Wczytywanie danych (MNIST)

In [ ]:
from tensorflow.keras import datasets
from sklearn.model_selection import train_test_split

(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
h, w = x_train.shape[1:]

x_train = x_train.reshape(x_train.shape[0], h, w, 1)
x_test = x_test.reshape(x_test.shape[0], h, w, 1)
input_shape = (h, w, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=10000, random_state=42)

(x_train.shape, y_train.shape), (x_val.shape, y_val.shape), (x_test.shape, y_test.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(x_train[0].squeeze(-1))
plt.title(y_train[0]);

In [ ]:
import numpy as np

print("{} unique labels.".format(np.unique(y_train)))

## 1. LeNet

Na początku zdefiniujemy klasyczny model LeNet-5 wprowadzony przez Yann Le Cun in 1998 ([url](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf)). Model jest na tyle prosty, że możemy go łatwo zdefiniować z wykorzystaniem **Sequential** API.

![lenet archi](images/lenet.png)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AvgPool2D, MaxPool2D, Dense, Flatten
from tensorflow.keras import optimizers

lenet = Sequential(name="LeNet-5")

lenet.add(Conv2D(6, kernel_size=(5, 5), activation="tanh", padding="same", input_shape=input_shape, name="C1"))
lenet.add(MaxPool2D(pool_size=(2, 2), name="S2"))
lenet.add(Conv2D(16, kernel_size=(5, 5), activation='tanh', name="C3"))
lenet.add(AvgPool2D(pool_size=(2, 2), name="S4"))
lenet.add(Conv2D(120, kernel_size=(5, 5), activation='tanh', name="C5"))
lenet.add(Flatten())
lenet.add(Dense(84, activation='tanh', name="F6"))
lenet.add(Dense(10, activation='softmax'))

lenet.summary()

In [ ]:
n_epochs = 5
batch_size = 256

lenet.compile(
    optimizer=optimizers.SGD(lr=0.1),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

lenet.fit(
    x_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(x_val, y_val)
)

In [ ]:
lenet.evaluate(x_test, y_test, verbose=1)

Uwaga: LeNet został na początku zdefiniowany z wykorzystaniem funkcji aktywacji `tanh` lub `sigmoid`, aktualnie
te funkcje aktywacji są rzadko używane. Można pokazać, że obie funkcje "nasycają się" (saturate)
dla bardzo małych i bardzo dużych wartości co powoduje "zanik" gradientu.

Dlatego aktualnie większość sieci wykorzystuje `ReLU` w warstwach ukrytych lub inne o podobnych własnościach
(https://keras.io/layers/advanced-activations/).


## 2. Inception

Model "Inception" został wprowadzony w 2014 przez Szegedy et al. ([paper url](https://arxiv.org/abs/1409.4842)).

Konwolucje mają efektywne "pole chłonności" (receptive field): im większe "kernels" i im głębszy (deeper) model, tym więcej pikseli obrazu *zobaczymy*. Dobrze wyjaśnienia to: [medium blog](https://medium.com/mlreview/a-guide-to-receptive-field-arithmetic-for-convolutional-neural-networks-e0f514068807).

W modelu tym, różne "convolution kernels" są ze sobą łączone. Mniejsze "kernels" wykorzystywane są do mniejszych
klastrów cech (można myśleć o tym jak o detalach np. oko), podchas duże "kernels" dla większych kastrów cech
(np. twarz)

![inception archi](images/inception.png)

Tym razem wykorzystamy **Functional** API  do zdefiniowania pojedynczej warstwy "Inception layer"

Przykład:

```python
a = Input(shape=(32,))
b = Dense(32)(a)
model = Model(inputs=a, outputs=b)
```

In [ ]:
from tensorflow.keras.layers import Concatenate, Input
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Concatenate, Input
from tensorflow.keras.models import Model


def inception_layer(tensor, n_filters):
    branch1x1 = Conv2D(n_filters, kernel_size=(1, 1), activation="relu", padding="same")(tensor)
    branch5x5 = Conv2D(n_filters, kernel_size=(5, 5), activation="relu", padding="same")(tensor)
    branch3x3 = Conv2D(n_filters, kernel_size=(3, 3), activation="relu", padding="same")(tensor)

    branch_pool = MaxPool2D(pool_size=(3, 3), strides=(1, 1), padding="same")(tensor)

    output = Concatenate(axis=-1)(
        [branch1x1, branch5x5, branch3x3, branch_pool]
    )
    return output


input_tensor = Input(shape=input_shape)
x = Conv2D(16, kernel_size=(5, 5), padding="same")(input_tensor)
x = inception_layer(x, 32)
x = Flatten()(x)
output_tensor = Dense(10, activation="softmax")(x)

mini_inception = Model(inputs=input_tensor, outputs=output_tensor)

mini_inception.summary()

In [ ]:
n_epochs = 5
batch_size = 256

mini_inception.compile(
    optimizer=optimizers.SGD(lr=0.1),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

mini_inception.fit(
    x_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(x_val, y_val)
)

In [ ]:
mini_inception.evaluate(x_test, y_test, verbose=1)

## 3. ResNet

ResNet (*Residual Networks*) model został wprowadzony przez He et al. in 2015 ([paper url](https://arxiv.org/abs/1512.03385)). Autorzy pracy zauważyli, że więcej warstw znacznie poprawia wydajność sieci, niestety
bardzo trudno uczyć takie sieci (backpropagate the gradients).

Trik który pozwalał uczyć takie sieci umożliwiając łatwiejszy "przepływ" gradientu polega na wprowadzeniu skrótów

![resnet archi](images/resnet.png)

Tym razem wykorzystamy **Oriented-Object** API:

Przykład:
```python
class MyModel(Model):
    def __init__(self):
        self.classifier = Dense(10, activation="softmax")
        
    def call(self, inputs):
        return self.classifier(inputs)
```

In [ ]:
from tensorflow.keras.layers import Add, Layer, Activation


class ResidualBlock(Layer):
    def __init__(self, n_filters):
        super().__init__(name="ResidualBlock")

        self.conv1 = Conv2D(n_filters, kernel_size=(3, 3), activation="relu", padding="same")
        self.conv2 = Conv2D(n_filters, kernel_size=(3, 3), padding="same")
        self.add = Add()
        self.last_relu = Activation("relu")

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(inputs)

        y = self.add([x, inputs])
        y = self.last_relu(y)

        return y


class MiniResNet(Model):
    def __init__(self, n_filters):
        super().__init__()

        self.conv = Conv2D(n_filters, kernel_size=(5, 5), padding="same")
        self.block = ResidualBlock(n_filters)
        self.flatten = Flatten()
        self.classifier = Dense(10, activation="softmax")

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.block(x)
        x = self.flatten(x)
        y = self.classifier(x)

        return y


mini_resnet = MiniResNet(32)
mini_resnet.build((None, *input_shape))
mini_resnet.summary()


In [ ]:
n_epochs = 5
batch_size = 256

mini_resnet.compile(
    optimizer=optimizers.SGD(lr=0.1),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

mini_resnet.fit(
    x_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(x_val, y_val)
)

In [ ]:
mini_resnet.evaluate(x_test, y_test, verbose=1)